In [1]:
import numpy as np
from itertools import product,combinations
import matplotlib.pyplot as plt

In [2]:
epsilon = 1e-2
delta = 0.01
m = 100
gen_max = 300
dim = 2
mutation_factor=0.1
crossover_rate=0.5
seed = 0

# how many parts/slices do you desire in each dimension?
parts = 100

def system_eq(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    return np.array([f1,f2])

boundaries = np.array([(-10,10) for _ in range (dim)])

In [3]:
def objective_function(x:np.ndarray):
    F_array = system_eq(x)
    denom = 0
    for f in F_array:
        denom +=np.abs(f)
    F = 1/(1+denom)
    return -F

In [4]:
def slice_hypercube(lower_bounds, upper_bounds, interval, dim):
    # Create a list of arrays, each containing points spaced h apart for each dimension
    points = [np.arange(lower_bounds[i], upper_bounds[i], interval) for i in range(dim)]
    
    # Use meshgrid to create a grid of points in n-dimensional space
    grids = np.meshgrid(*points, indexing='ij')
    
    # Flatten and combine the grid points into a single 2D array
    grid_points = np.vstack([grid.ravel() for grid in grids]).T
    
    # Generate all vertices for smaller hypercubes within each grid cell
    return np.array([grid_points + offset for offset in product([0, interval], repeat=dim)])

In [5]:
# length of each parts in each dimension
inc_int = (boundaries[0,1]-boundaries[0,0])/parts

# Hypercubes
hypercubes_edges = slice_hypercube(lower_bounds=boundaries[:,0],
                    upper_bounds=boundaries[:,1],
                    interval=inc_int,
                    dim=dim)

In [6]:
cluster = []
for hypercube_id in range(hypercubes_edges.shape[1]):
    X0 = hypercubes_edges[:,hypercube_id,:]
    F_list = system_eq(X0.T)

    # cek jika f yang berubah tanda dari F_list jika dievaluasi di tiap edge hypercube
    product_combination = np.array([[a*b for a,b in combinations(F_list[i],2)] for i in range (F_list.shape[0])])

    # jika semua f dari F_list berubah tanda jika dievaluasi di tiap edge hypercube, maka ada akar di situ
    change_sign = np.array([np.any(product_combination[i]<0) for i in range (product_combination.shape[0])])
    if np.all(change_sign==True):
        # print(f'Ada akar di sini: \nX0={X0}')
        cluster.append(X0)

cluster = np.array(cluster)
print(f"Number of clusters containing root: {cluster.shape[0]}")

Number of clusters containing root: 12


# DE Evaluation

In [7]:
import importlib
import re

import sys
module_path = r"D:\OneDrive - Institut Teknologi Bandung\[AKADEMIK]\Semester 7-8\TA\Thesis\Differential Evolution"
sys.path.append(module_path)
import deal as dl
importlib.reload(dl)

sys.path.remove(module_path)


In [8]:
roots = []
values = []
for i in range (cluster.shape[0]):
    subbound = np.array([[cluster[i,:,:][:,d].min(),cluster[i,:,:][:,d].max()] for d in range(cluster.shape[2])])
    root,value = dl.differensial_evolution(objective_func=objective_function,
                                           population_size=m,
                                           boundaries=subbound, 
                                           gen_max=gen_max, 
                                           mutation_factor=mutation_factor, 
                                           crossover_rate=crossover_rate,
                                           seed=0,
                                           print_gen = True,
                                           history = False)
    roots.append(root)
    values.append(value)

archive = np.array(roots)
scores = np.array(values)
print(f'Number of Clusters containing root: {cluster.shape[0]}\n')
print(f'Roots:\n{archive}\n\nValues: \n{values}')

=========Generation 0=========
Best Point:[-6.4478125  0.1553125] with score -0.9866137684079332
=========Generation 1=========
Best Point:[-6.4478125  0.1553125] with score -0.9866137684079332
=========Generation 2=========
Best Point:[-6.44306484  0.15484406] with score -0.9890806559463396
=========Generation 3=========
Best Point:[-6.44306484  0.15484406] with score -0.9890806559463396
=========Generation 4=========
Best Point:[-6.44306484  0.15484406] with score -0.9890806559463396
=========Generation 5=========
Best Point:[-6.43234761  0.15577727] with score -0.9907942062649057
=========Generation 6=========
Best Point:[-6.4359204   0.15544812] with score -0.9977567719681275
=========Generation 7=========
Best Point:[-6.4359204   0.15544812] with score -0.9977567719681275
=========Generation 8=========
Best Point:[-6.43756855  0.15532129] with score -0.9993517982990474
=========Generation 9=========
Best Point:[-6.4369469   0.15535034] with score -0.999747211167121
=========Genera

In [9]:
"""Choosing Best Solution"""
if dim == 1:
    list_criteria = [element for sublist in archive for element in sublist] #convert from 2D array into 1D array
else:
    list_criteria = archive
eligible_roots = np.array([x for x in list_criteria if (objective_function(x))<-1+epsilon])
id_duplicated_roots = []
for i in range(len(eligible_roots)):
    for j in range (i+1,len(eligible_roots)):
        if np.linalg.norm(eligible_roots[i]-eligible_roots[j])<delta:
            id_duplicated_roots.append([i,j])
id_duplicated_roots = np.unique(id_duplicated_roots,axis=0)
deselected_id_duplicated_roots = []
for i in range (len(id_duplicated_roots)):
    root_a = objective_function(eligible_roots[id_duplicated_roots[i][0]])
    root_b = objective_function(eligible_roots[id_duplicated_roots[i][1]])
    if root_a<=root_b:
        id_duplicated_root = id_duplicated_roots[i][1]
    else:
        id_duplicated_root = id_duplicated_roots[i][0]
    deselected_id_duplicated_roots.append(id_duplicated_root)

if deselected_id_duplicated_roots:
    unique_roots = np.ones(len(eligible_roots),dtype=bool)
    unique_roots[deselected_id_duplicated_roots] = False
    final_root = eligible_roots[unique_roots]
else:
    final_root = eligible_roots
print(final_root)

[[-6.43716259  0.15534788]
 [-6.1171134  -0.16347552]
 [-0.93212205  1.06787439]
 [-0.15528339  6.43983572]
 [ 0.16333346  6.12243408]
 [ 0.6671212   0.6901032 ]]
